In [2]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import requests
import urllib3

In [3]:
#Setup
requests.packages.urllib3.disable_warnings()
requests.packages.urllib3.util.ssl_.DEFAULT_CIPHERS += ':HIGH:!DH:!aNULL'
try:
    requests.packages.urllib3.contrib.pyopenssl.util.ssl_.DEFAULT_CIPHERS += ':HIGH:!DH:!aNULL'
except AttributeError:
    # no pyopenssl support used / needed / available
    pass

In [4]:
Link = "https://ic2s2.mit.edu/program"
req = requests.get(Link, verify= False)
soup = BeautifulSoup(req.content, features="html.parser")
#Ren træning, kunne bare kopiere det rigtige link
text = soup.find("div", {"class": "article-content"})
str_text = str(text).split("src=")
docs_link = str_text[-1].split(" ")[0][1:-1]

In [5]:
r = requests.get(docs_link)
soup = BeautifulSoup(r.content, features="html.parser")
table = soup.find("table", {"class": "waffle"})
table_rows = table.find_all("tr")
ths = table_rows[0].find_all('th')
header = [th.text.replace("\n","") for th in ths]

#HERE I GET THE ROWS
rows = []
for tr in table_rows[1:]:
    tds = tr.find_all('td')
    row = [td.text.replace("\n","") for td in tds]
    rows.append(row)

In [51]:
df = pd.DataFrame(rows, columns=header[0:5])
names_plus = [name.split(', ') for name in list(df.iloc[:,2][1:])]
names = []
for name in names_plus:
    for n in name:
        if len(n) :
            names.append(n)
print(len(names), len(set(names)))
science_people = list(set(names))


1389 990


In [63]:
#setup
BASE_URL = "https://api.semanticscholar.org/graph/"
VERSION  = "v1/"
RESOURCE = "author/search?query="
complete_url = BASE_URL + VERSION + RESOURCE

In [62]:
authorIds = dict()
for i, name in enumerate(science_people):
    name = name.replace(" ","+").lower()
    response = requests.get(complete_url + name).json()
    if len(response["data"]):
        authorIds[(response["data"][0]['authorId'])] = list()
    if i == 5: break
authorIds

{'101236378': [],
 '144212002': [],
 '1576739598': [],
 '2331299': [],
 '1956650': []}

In [77]:
for i, id in enumerate(authorIds.keys()):
    colabs = set()
    full_url = BASE_URL + VERSION + "author/" + id + "?" + "fields=papers.authors"
    response = requests.get(full_url).json()
    for paper in response["papers"]:
        for c_id in paper["authors"]:
            colabs.add(c_id["authorId"])
    authorIds[id] = colabs - set(id)
authorIds

{'101236378': {'101236378',
  '101748268',
  '108066828',
  '118597836',
  '118842067',
  '120500809',
  '1379672188',
  '1422489633',
  '143604157',
  '143748180',
  '144651240',
  '144787428',
  '144983715',
  '145641729',
  '2028268285',
  '2075454246',
  '2112614469',
  '2118114767',
  '30346473',
  '32107505',
  '38316621',
  '40561274',
  '49637716',
  '50218504',
  '51056157',
  '66392671',
  '70422778',
  '70454356',
  '72610419',
  '80432710',
  '98601449',
  '98822172'},
 '144212002': {'103833054',
  '10769465',
  '113011351',
  '113211858',
  '113914651',
  '114460984',
  '114709659',
  '115010546',
  '115068402',
  '115609720',
  '115850573',
  '117319147',
  '117889856',
  '118201627',
  '1404609254',
  '1404876257',
  '143912271',
  '143974140',
  '144088223',
  '144212002',
  '145306709',
  '145380101',
  '145687960',
  '152114511',
  '152823992',
  '153001151',
  '153406797',
  '153823344',
  '1700820',
  '2050682428',
  '2053795092',
  '2055665491',
  '2069545921',
  '